# SkySat-snow pipeline

## Define settings and paths in directory

In [2]:
import os
import sys

# Define paths in directory
site_name = "Banner"
date = "20240419-1"
code_dir = '/Users/raineyaberle/Research/PhD/SnowDEMs/skysat-snow'
asp_dir = '/Users/raineyaberle/Research/PhD/SnowDEMs/StereoPipeline-3.5.0-alpha-2024-10-05-x86_64-OSX/bin'
ref_dem_fn = f'/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/study-sites/{site_name}/refdem/{site_name}_REFDEM_WGS84.tif'
pc_fn = f'/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/study-sites/{site_name}/{date}/{site_name}_{date}_pc_merged.laz'
roads_fn = '/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/ITD_Functional_Class/ITD_HWY_21.shp'
multispec_dir = f'/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/study-sites/{site_name}/{date}/{site_name}_{date}_4band_mosaic.tif'
out_dir = f'/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/study-sites/{site_name}/{date}'
res = 2 # spatial resolution of outputs [m]
roads_buffer = 5 # buffer for roads geospatial file [m]

# Check that input files and directories exist
if not os.path.exists(ref_dem_fn):
    print('Reference DEM file not found, please correct ref_dem_fn before continuing.')
if not os.path.exists(pc_fn):
    print('Point cloud file not found, please correct pc_fn before continuing.')
if not os.path.exists(roads_fn):
    print('Roads geospatial file not found, please correct roads_fn before continuing.')
if not os.path.exists(multispec_dir):
    print('Multispectral images folder not found, please correct multispec_dir before continuing.')
if not os.path.exists(asp_dir):
    print('Path to ASP not found, please correct asp_dir before continuing.')
out_dir = os.path.join(out_dir, 'skysat_snow')
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
    print('Made directory for outputs:', out_dir)

# Make results directories
preprocess_dir = os.path.join(out_dir, 'preprocess')
masks_dir = os.path.join(out_dir, 'land_cover_masks')
corr_coreg_diff_dir = os.path.join(out_dir, 'corr_coreg_diff')
analysis_dir = os.path.join(out_dir, 'ddem_analysis')
for folder in [preprocess_dir, masks_dir, corr_coreg_diff_dir, analysis_dir]:
    if not os.path.exists(folder):
        os.mkdir(folder)

# Add path to pipeline utilities
sys.path.append(os.path.join(code_dir, 'scripts'))
import pipeline_utils as f

## Run pipeline

In [ ]:
##### 1. PREPROCESS THE POINT CLOUDS #####
print('\n1. PREPROCESS THE POINT CLOUDS')
# Merge and filter point clouds
pc_filtered_las_fn, pc_filtered_tif_fn = f.preprocess_point_clouds(pc_fn, ref_dem_fn, res, preprocess_dir)
# Mosaic 4-band SR imagery, preprocess roads and reference DEM
multispec_mosaic_fn, roads_adj_fn, ref_dem_adj_fn = f.preprocess_multispec_refdem_roads(multispec_dir, ref_dem_fn, roads_fn, roads_buffer, res, out_dir)


##### 2. CONSTRUCT LAND COVER MASKS #####
print('\n2. CONSTRUCT LAND COVER MASKS')
multispec_mosaic_fn, trees_mask_fn, snow_mask_fn, roads_mask_fn, ss_mask_fn = f.construct_land_cover_masks(multispec_mosaic_fn, 
                                                                                                           roads_adj_fn, 
                                                                                                           masks_dir, 
                                                                                                           roads_buffer,
                                                                                                           ndvi_threshold=0.1,
                                                                                                           ndsi_threshold=0.0)


##### 3. CORRECT, COREGISTER, AND DIFFERENCE DEM #####
print('\n3. CORRECT, COREGISTER, AND DIFFERENCE DEM')
final_dem_fn, final_ddem_fn = f.correct_coregister_difference(pc_filtered_tif_fn, ref_dem_adj_fn, ss_mask_fn, roads_mask_fn, corr_coreg_diff_dir, plot_results=True)


##### 4. PLOT FINAL DEM AND dDEM #####
print('\n4. PLOT THE FINAL DEM AND dDEM')
f.plot_dem_ddem(final_dem_fn, final_ddem_fn, roads_mask_fn, os.path.join(out_dir, 'final_dem_ddem.png'), vmin=-5, vmax=5)


##### 5. ANALYZE dDEM #####
print('\n5. ANALYZE dDEM VS. LAND COVER TYPE AND TERRAIN PARAMETERS')
f.analyze_ddem(final_ddem_fn, ref_dem_adj_fn, roads_mask_fn, snow_mask_fn, ss_mask_fn, trees_mask_fn, analysis_dir)


## Apply terrain correction

In [ ]:
# import xdem
# import matplotlib.pyplot as plt
# import geoutils as gu

# print('Loading input files...')
# dem = xdem.DEM(final_dem_fn)
# ref_dem = xdem.DEM(ref_dem_adj_fn)
# ref_dem = ref_dem.reproject(dem)
# ss_mask = gu.Raster(ss_mask_fn, load_data=True)
# ss_mask = ss_mask.reproject(dem)
# ss_mask = (ss_mask==1)

# # Test several terrain variables
# terrain_vars = ['elevation', 'slope', 'aspect', 'maximum_curvature']
# for terrain_var in terrain_vars:
#     print(terrain_var)
#     terrain_bias = xdem.coreg.TerrainBias(terrain_var).fit(ref_dem, dem, ss_mask)
#     dem_corrected = terrain_bias.apply(dem)
#     ddem_before = dem - ref_dem
#     ddem_after = dem_corrected - ref_dem
#     fig, ax = plt.subplots(1, 2, figsize=(12,6))
#     ddem_before.plot(cmap='coolwarm_r', vmin=-5, vmax=5, ax=ax[0])
#     ax[0].set_title('dDEM')
#     ddem_after.plot(cmap='coolwarm_r', vmin=-5, vmax=5, ax=ax[1])
#     ax[1].set_title('Corrected dDEM')
#     plt.suptitle(terrain_var)
#     plt.show()

## Try correcting for directional bias

In [ ]:
# import numpy as np

# for angle in np.linspace(0, 360, 5)[0:-1]:
#     dir_bias = xdem.coreg.DirectionalBias(angle=angle).fit(ref_dem, dem, ss_mask)
#     dem_corrected = dir_bias.apply(dem)
#     ddem = dem_corrected - ref_dem
#     fig, ax = plt.subplots(1, 1, figsize=(6,6))
#     ddem.plot(cmap='coolwarm_r', vmin=-5, vmax=5, ax=ax)
#     ax.set_title(angle)
#     plt.show()

In [ ]:
# deramp = xdem.coreg.Deramp().fit(ref_dem, dem, ss_mask)
# dem_corrected = deramp.apply(dem)

# ddem = dem_corrected - ref_dem

# # roads_mask = gu.Raster(roads_mask_fn)
# # roads_mask = roads_mask.reproject(dem)
# # roads_mask = (roads_mask==1)
# # ddem_roads = ddem[roads_mask]
# # ddem_roads_med = np.nanmedian(ddem_roads.data)
# # print(ddem_roads_med)

# # ddem -= ddem_roads_med

# ddem_ss = ddem[ss_mask]
# ddem_ss_med = np.nanmedian(ddem_ss.data)
# print(ddem_ss_med)

# ddem.plot(cmap='coolwarm_r', vmin=-5, vmax=5)
# plt.show()